In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/brain-tumor-classification-mri/Training/no_tumor/image(84).jpg
/kaggle/input/brain-tumor-classification-mri/Training/no_tumor/image(44).jpg
/kaggle/input/brain-tumor-classification-mri/Training/no_tumor/image(245).jpg
/kaggle/input/brain-tumor-classification-mri/Training/no_tumor/6.jpg
/kaggle/input/brain-tumor-classification-mri/Training/no_tumor/image(238).jpg
/kaggle/input/brain-tumor-classification-mri/Training/no_tumor/image(196).jpg
/kaggle/input/brain-tumor-classification-mri/Training/no_tumor/image(108).jpg
/kaggle/input/brain-tumor-classification-mri/Training/no_tumor/image(310).jpg
/kaggle/input/brain-tumor-classification-mri/Training/no_tumor/image (5).jpg
/kaggle/input/brain-tumor-classification-mri/Training/no_tumor/image(186).jpg
/kaggle/input/brain-tumor-classification-mri/Training/no_tumor/image(29).jpg
/kaggle/input/brain-tumor-classification-mri/Training/no_tumor/image(140).jpg
/kaggle/input/brain-tumor-classification-mri/Training/no_tumor/image(224).jpg

BRAIN TUMOR CLASSIFICATION PROJECT - SMARTINTERNZ

In [2]:
# Data Augmentation

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=(1./255),horizontal_flip=True,shear_range=0.2)
test_gen = ImageDataGenerator(rescale=(1./255))  #--> (0 to 255) convert to (0 to 1)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
train = train_gen.flow_from_directory('/kaggle/input/brain-tumor-classification-mri/Training',
                                      target_size=(120, 120),
                                      class_mode='categorical', 
                                      batch_size=15)
test = test_gen.flow_from_directory('/kaggle/input/brain-tumor-classification-mri/Testing',
                                    target_size=(120, 120),
                                      class_mode='categorical', 
                                      batch_size=15)

Found 2870 images belonging to 4 classes.
Found 394 images belonging to 4 classes.


In [4]:
train.class_indices

{'glioma_tumor': 0, 'meningioma_tumor': 1, 'no_tumor': 2, 'pituitary_tumor': 3}

In [5]:
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Convolution2D(12,(3,3),activation='relu',input_shape=(120, 120, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(24,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(36,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(62,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(4,activation='softmax'))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 118, 118, 12)      336       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 59, 59, 12)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 57, 24)        2616      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 28, 28, 24)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 26, 26, 36)        7812      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 36)       0

In [7]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [9]:
model.fit(train,batch_size=15,validation_data=test,epochs=10)

Epoch 1/10
192/192 [==============================] - 32s 168ms/step - loss: 0.8785 - accuracy: 0.6153 - val_loss: 2.4141 - val_accuracy: 0.3528
Epoch 2/10
192/192 [==============================] - 17s 88ms/step - loss: 0.6291 - accuracy: 0.7404 - val_loss: 2.7519 - val_accuracy: 0.4797
Epoch 3/10
192/192 [==============================] - 16s 85ms/step - loss: 0.5113 - accuracy: 0.7930 - val_loss: 2.0002 - val_accuracy: 0.5000
Epoch 4/10
192/192 [==============================] - 17s 88ms/step - loss: 0.4142 - accuracy: 0.8272 - val_loss: 2.6724 - val_accuracy: 0.4848
Epoch 5/10
192/192 [==============================] - 16s 85ms/step - loss: 0.3684 - accuracy: 0.8488 - val_loss: 2.6636 - val_accuracy: 0.5761
Epoch 6/10
192/192 [==============================] - 17s 88ms/step - loss: 0.3177 - accuracy: 0.8770 - val_loss: 2.7164 - val_accuracy: 0.6066
Epoch 7/10
192/192 [==============================] - 17s 88ms/step - loss: 0.2703 - accuracy: 0.8993 - val_loss: 2.4509 - val_accuracy

The above is the basic code... now we try to hypertune the parameters and find the best parameters for the model

Trying pre-trained model

In [10]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np

In [12]:
TL_train = train_gen.flow_from_directory("/kaggle/input/brain-tumor-classification-mri/Training",
                                      target_size=(224,224),
                                      batch_size=15,
                                      class_mode='categorical')

TL_test = test_gen.flow_from_directory('/kaggle/input/brain-tumor-classification-mri/Testing',
                                      target_size=(224,224),
                                      batch_size=15,
                                      class_mode='categorical')

Found 2870 images belonging to 4 classes.
Found 394 images belonging to 4 classes.


In [13]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [29]:
vgg = VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))

for layer in vgg.layers:
  layer.trainable=False

x = Flatten()(vgg.output)

prediction = Dense(4,activation='softmax')(x)

vggmodel = Model(inputs=vgg.input,outputs=prediction)

vggmodel.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [30]:
vggmodel.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [31]:
vggmodel.fit_generator(TL_train,validation_data=TL_test,epochs=15,steps_per_epoch=len(TL_train),
                    validation_steps=len(TL_test))

/tmp/ipykernel_29/124170498.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  vggmodel.fit_generator(TL_train,validation_data=TL_test,epochs=15,steps_per_epoch=len(TL_train),


Epoch 1/15
192/192 [==============================] - 47s 227ms/step - loss: 0.6679 - accuracy: 0.7519 - val_loss: 1.6589 - val_accuracy: 0.6447
Epoch 2/15
192/192 [==============================] - 41s 214ms/step - loss: 0.3507 - accuracy: 0.8679 - val_loss: 1.8092 - val_accuracy: 0.6066
Epoch 3/15
192/192 [==============================] - 42s 217ms/step - loss: 0.2442 - accuracy: 0.9028 - val_loss: 2.3300 - val_accuracy: 0.7005
Epoch 4/15
192/192 [==============================] - 42s 216ms/step - loss: 0.1671 - accuracy: 0.9362 - val_loss: 2.4385 - val_accuracy: 0.7030
Epoch 5/15
192/192 [==============================] - 41s 213ms/step - loss: 0.1503 - accuracy: 0.9422 - val_loss: 2.5530 - val_accuracy: 0.7360
Epoch 6/15
192/192 [==============================] - 42s 217ms/step - loss: 0.1026 - accuracy: 0.9669 - val_loss: 2.6585 - val_accuracy: 0.7208
Epoch 7/15
192/192 [==============================] - 42s 216ms/step - loss: 0.0915 - accuracy: 0.9700 - val_loss: 2.3939 - val_ac

In [49]:
img1 = image.load_img('/kaggle/input/brain-tumor-classification-mri/Training/glioma_tumor/gg (15).jpg',target_size=(224,224))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred = np.argmax(vggmodel.predict(img1))
print(pred)
output = ['glioma', 'meningioma', 'no_tumor', 'pituitary_tumor']
print(output[pred])

1/1 [==============================] - 0s 20ms/step
1
meningioma


In [74]:
from tqdm import tqdm
labels = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
for i in labels:
    folderPath = os.path.join('/kaggle/input/brain-tumor-classification-mri/','Training/',i)
    cnt=0
    acc=0
    for j in tqdm(os.listdir(folderPath)):
        img = image.load_img(os.path.join(folderPath,j),target_size=(224,224))
        img = image.img_to_array(img)
        img = np.expand_dims(img,axis=0)
        pred = np.argmax(vggmodel.predict(img))
        if i == labels[pred]:
            acc+=1;
        cnt+=1;
    print("Accuracy for the Class "+i+" is "+str((acc/cnt))+"%")

  0%|          | 0/826 [00:00<?, ?it/s]

1/1 [==============================] - 0s 20ms/step


  0%|          | 2/826 [00:00<00:58, 14.07it/s]

1/1 [==============================] - 0s 23ms/step


  0%|          | 4/826 [00:00<00:59, 13.91it/s]

...

1/1 [==============================] - 0s 23ms/step


100%|█████████▉| 825/827 [01:02<00:00, 13.28it/s]

1/1 [==============================] - 0s 23ms/step


100%|██████████| 827/827 [01:02<00:00, 13.24it/s]

Accuracy for the Class pituitary_tumor is 0.10519951632406288%


VGG19

In [51]:
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input

In [52]:
vgg19 = VGG19(include_top=False,weights='imagenet',input_shape=(224,224,3))

for layer in vgg19.layers:
  layer.trainable=False

x = Flatten()(vgg19.output)

prediction = Dense(4,activation='softmax')(x)

vgg19model = Model(inputs=vgg19.input,outputs=prediction)

vgg19model.summary()

80134624/80134624 [==============================] - 0s 0us/step
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

In [53]:
vgg19model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [54]:
vgg19model.fit_generator(TL_train,validation_data=TL_test,epochs=15,steps_per_epoch=len(TL_train),
                    validation_steps=len(TL_test))

/tmp/ipykernel_29/4005773052.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  vgg19model.fit_generator(TL_train,validation_data=TL_test,epochs=15,steps_per_epoch=len(TL_train),


Epoch 1/15
192/192 [==============================] - 46s 233ms/step - loss: 0.7337 - accuracy: 0.7293 - val_loss: 2.0576 - val_accuracy: 0.5990
Epoch 2/15
192/192 [==============================] - 43s 222ms/step - loss: 0.3715 - accuracy: 0.8561 - val_loss: 2.2503 - val_accuracy: 0.6878
Epoch 3/15
192/192 [==============================] - 43s 221ms/step - loss: 0.2893 - accuracy: 0.8958 - val_loss: 2.1071 - val_accuracy: 0.7081
Epoch 4/15
192/192 [==============================] - 43s 223ms/step - loss: 0.2779 - accuracy: 0.8920 - val_loss: 1.9485 - val_accuracy: 0.7157
Epoch 5/15
192/192 [==============================] - 42s 221ms/step - loss: 0.1697 - accuracy: 0.9376 - val_loss: 2.9626 - val_accuracy: 0.5914
Epoch 6/15
192/192 [==============================] - 43s 222ms/step - loss: 0.1485 - accuracy: 0.9495 - val_loss: 2.3247 - val_accuracy: 0.6929
Epoch 7/15
192/192 [==============================] - 43s 223ms/step - loss: 0.1641 - accuracy: 0.9411 - val_loss: 2.9850 - val_ac

In [66]:
img1 = image.load_img('/kaggle/input/brain-tumor-classification-mri/Testing/pituitary_tumor/image(94).jpg',target_size=(224,224))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred = np.argmax(vgg19model.predict(img1))
print(pred)
output = ['glioma', 'meningioma', 'no_tumor', 'pituitary_tumor']
print(output[pred])

1/1 [==============================] - 0s 19ms/step
2
no_tumor


In [73]:
from tqdm import tqdm
labels = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
for i in labels:
    folderPath = os.path.join('/kaggle/input/brain-tumor-classification-mri/','Training/',i)
    cnt=0
    acc=0
    for j in tqdm(os.listdir(folderPath)):
        img = image.load_img(os.path.join(folderPath,j),target_size=(224,224))
        img = image.img_to_array(img)
        img = np.expand_dims(img,axis=0)
        pred = np.argmax(vgg19model.predict(img))
        if i == labels[pred]:
            acc+=1;
        cnt+=1;
    print("Accuracy for the Class "+i+" is "+str((acc/cnt))+"%")

  0%|          | 0/826 [00:00<?, ?it/s]

1/1 [==============================] - 0s 22ms/step


  0%|          | 2/826 [00:00<01:01, 13.36it/s]

1/1 [==============================] - 0s 21ms/step


  0%|          | 4/826 [00:00<01:01, 13.37it/s]

...

1/1 [==============================] - 0s 23ms/step


100%|█████████▉| 825/827 [01:04<00:00, 12.57it/s]

1/1 [==============================] - 0s 24ms/step


100%|██████████| 827/827 [01:05<00:00, 12.70it/s]

Accuracy for the Class pituitary_tumor is 0.0%


TRYING RESNET !

In [17]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [18]:
resnet = ResNet50(include_top=False,input_shape=(224,224,3))

for layer in resnet.layers:
  layer.trainable=False

x = Flatten()(resnet.output)

out = Dense(4, activation='softmax')(x)

res_model = Model(inputs=resnet.input,outputs=out)

res_model.summary()

94765736/94765736 [==============================] - 0s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                           

In [25]:
res_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [27]:
res_model.fit(TL_train,epochs=10,validation_data=TL_test,steps_per_epoch=len(TL_train),
              validation_steps=len(TL_test))

Epoch 1/10
192/192 [==============================] - 46s 233ms/step - loss: 2.1283 - accuracy: 0.5174 - val_loss: 4.5111 - val_accuracy: 0.3680
Epoch 2/10
192/192 [==============================] - 41s 212ms/step - loss: 1.0307 - accuracy: 0.6533 - val_loss: 4.3876 - val_accuracy: 0.3909
Epoch 3/10
192/192 [==============================] - 41s 211ms/step - loss: 1.1186 - accuracy: 0.6638 - val_loss: 4.8536 - val_accuracy: 0.4365
Epoch 4/10
192/192 [==============================] - 41s 213ms/step - loss: 0.9183 - accuracy: 0.7014 - val_loss: 8.5131 - val_accuracy: 0.2919
Epoch 5/10
192/192 [==============================] - 41s 211ms/step - loss: 1.0650 - accuracy: 0.6805 - val_loss: 4.9759 - val_accuracy: 0.5305
Epoch 6/10
192/192 [==============================] - 41s 211ms/step - loss: 0.8346 - accuracy: 0.7279 - val_loss: 6.0522 - val_accuracy: 0.3858
Epoch 7/10
192/192 [==============================] - 41s 214ms/step - loss: 0.8964 - accuracy: 0.7286 - val_loss: 4.5809 - val_ac

In [75]:
from tqdm import tqdm
labels = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
for i in labels:
    folderPath = os.path.join('/kaggle/input/brain-tumor-classification-mri/','Training/',i)
    cnt=0
    acc=0
    for j in tqdm(os.listdir(folderPath)):
        img = image.load_img(os.path.join(folderPath,j),target_size=(224,224))
        img = image.img_to_array(img)
        img = np.expand_dims(img,axis=0)
        pred = np.argmax(res_model.predict(img))
        if i == labels[pred]:
            acc+=1;
        cnt+=1;
    print("Accuracy for the Class "+i+" is "+str((acc/cnt))+"%")

  0%|          | 0/826 [00:00<?, ?it/s]

1/1 [==============================] - 2s 2s/step


  0%|          | 1/826 [00:01<21:34,  1.57s/it]

1/1 [==============================] - 0s 28ms/step


  0%|          | 3/826 [00:01<06:21,  2.16it/s]

...

1/1 [==============================] - 0s 24ms/step


100%|█████████▉| 826/827 [01:01<00:00, 14.07it/s]

1/1 [==============================] - 0s 24ms/step


100%|██████████| 827/827 [01:01<00:00, 13.36it/s]

Accuracy for the Class pituitary_tumor is 0.0%
